# Class Scheduling Problem as a Constraint Satisfaction Problem (CSP) - Group 08
**This iPY notebook describes the implementation of a Class Scheduling Problem as a Constraint Satisfaction Problem (CSP), made by Group 08 (Augusto Pereira - 21136, Duarte Melo - 21149 and Nuno Veloso - 10411)**    
This implementation uses the AIMA (Artificial Intelligence: A Modern Approach) code, available on GitHub at https://github.com/aimacode/aima-python  


The group used the following files from AIMA code:
- csp.py
- utils.py
- search.py

In [ ]:
from lessons import Lesson
from csp import *
from my_utils import *
import time

# Problem formulation
The goal of this agent is to return a schedule that satisfies all the constraints specified, after receiving as input a set of data relative to classes, subjects and rooms.  
Some of the constraints are specified in the Project Guide, but the group decided to add even more to make the problem seem more realistic and similar to a real world problem.



## Variables
To be able to create constraints and return a solution, the group created a set variables, enabling us to create constraints and return a solution.
  
The "main" variable is the Lesson, but each Lesson is associated with a set of "sub-variables".

Each class has 10 Lessons, one "free day" (a day without lessons) and a "favourite room" (a room where the class has 2 to 4 lessons).  
Each Lesson has these sub-variables (in this case, Lesson nr. 0):  
- L0.c -> class that attends Lesson 0
- L0.su -> subject related to Lesson 0
- L0.d -> duration of Lesson 0
- L0.w -> weekday of Lesson 0
- L0.st -> start time of Lesson 0
- L0.r -> the room where Lesson 0 is attended

This is a small exemple of a result given by the script main.py, to explain how the result uses variables:
```

```{'L0.fd': 5, 'L0.fr': 1, 'L0.c': 1, 'L1.c': 1, 'L2.c': 1, 'L3.c': 1, 'L4.c': 1, 'L5.c': 1, 'L6.c': 1, 'L7.c': 1, 'L8.c': 1, 'L9.c': 1, 'L0.su': 3, 'L0.d': 2, 'L0.w': 4, 'L0.st': 8, 'L0.r': 1, 'L1.su': 3, 'L1.d': 2, 'L1.w': 2, 'L1.st': 10, 'L1.r': 2, 'L2.su': 4, 'L2.d': 2, 'L2.w': 4, 'L2.st': 10, 'L2.r': 1, 'L3.su': 2, 'L3.d': 2, 'L3.w': 3, 'L3.st': 12, 'L3.r': 1, 'L4.su': 4, 'L4.d': 2, 'L4.w': 6, 'L4.st': 10, 'L4.r': 0, 'L5.su': 5, 'L5.d': 2, 'L5.w': 2, 'L5.st': 12, 'L5.r': 2, 'L6.su': 1, 'L6.d': 2, 'L6.w': 3, 'L6.st': 10, 'L6.r': 2, 'L7.su': 1, 'L7.d': 2, 'L7.w': 2, 'L7.st': 8, 'L7.r': 2, 'L8.su': 2, 'L8.d': 2, 'L8.w': 3, 'L8.st': 8, 'L8.r': 2, 'L9.su': 5, 'L9.d': 2, 'L9.w': 6, 'L9.st': 8, 'L9.r': 0,```


As you can see, the class 1 has 10 Lessons (from 0 to 9).  
In the first lesson (L0), we assigned the fd - free day - and the fr - favourite room.

## Domains
For each variable, there is a domain - a set of values that can be assigned to this variable.  
For instance, the variable L.w (weekday) can only take values from 2 to 6, ie, days from Monday (2) to Friday (6).

In [ ]:
# each class has 2 to 4 lessons in a specific classroom
# to distribute "favourite" rooms per classes, we created a room_usages dictionary
# whenever we assign a room as a "favourite" room, the "usage" value of that room is incremented
# this ensures that rooms are assigned as favourite rooms in a balanced way
rooms_usages = {}
for room in rooms.keys():
    if room != 0:
        rooms_usages[room] = 0


# each class has 10 lessons per week
lessons_list = []
for x in range (len(classes)*10):
    # Lesson is a class that was created when we started coding this project
    # It is kinda irrelevant / unused at the moment
    new_l = Lesson(None, None, None, None, None, None)
    lessons_list.append(new_l)


# Domain defines the domain of each variable
domain = {}

# each class has 10 lessons
# Lesson 0-9 corresponds to Class 1
# Lesson 10-19 corresponds to Class 2...
# etc.
aux = 0
aux_final = 10
for x in classes:
    free_day = random.randint(2,6)

    room = least_used_room(rooms_usages)
    rooms_usages[room]+=1

    domain.update({f'L{aux}.fd': {free_day}}) # each class has a random day without lessons
    domain.update({f'L{aux}.fr': {room}}) # each class has a "favourite" room, where it has 2 to 4 lessons per week
    while aux != aux_final: 
        domain.update({f'L{aux}.c': {x}}) # assigning lessons to classes
        aux+=1
    
    aux = aux_final
    aux_final = aux+10


for index, list_el in enumerate(lessons_list):
    domain.update({f'L{index}.su': set(range(1,len(subjects)+1))}) # subjects domain assign
    domain.update({f'L{index}.d': {2}}) # duration domain assign 
    domain.update({f'L{index}.w': set(range(2,7))}) # weekday domain assign
    domain.update({f'L{index}.st': set(range(8,17))}) # start time domain assign
    domain.update({f'L{index}.r': set(range(0,len(rooms)+1))}) # rooms domain assign


As you can see, for each class "for x in classes:" we assigned a freeday (a random generated value between 2 and 6), a favourite room (here we created a rooms_usages dictionary to distribute rooms in a balaced way), and ten lessons (Class 1 - 0-9; Class 2 - 10-19; etc.)

Then, for each Lesson (regardless of the correspondent class), we set the "sub-variables" su (subject), d (duration), w (weekday), st (start time), and r (room).  
In order to assign the domains to subject and room variables we used the length of previously created dictionaries, that contain a number of items that may vary.  
These dictionaries will be explained in the section "Input Data".


## Input Data
The data received as input, used by the algorithm, is specified in three dictionaries.  
Each dictionary key will be used by the algorithm to return a schedule, as you have seen before in "Variables" section.

In [ ]:
# classes used in the algorithm, the name is irrelevant since only the numbers will be used
classes = {
    1: "LESI",
    2: "LESI-PL"
}

# subjects used in the algorithm, the algorithm accepts, at the moment, 5 subjects, the name is irrelevant
subjects =  {
    1: "Mobile",
    2: "Programming",
    3: "Artificial Intelligence",
    4: "Data Structures",
    5: "APIs"
}

# rooms used in the algorithm, the name is irrelevant but ROOM 0 = online!
rooms = {
    0: "Online",
    1: "Sala L",
    2: "Sala T",
    3: "Sala N"
}

## Constraints
To be able to return a good solution, that satisfies real life restrictions when creating a schedule (for example, a class can't be in two lessons at the same time), the group had to create Constraints that are passed to the algorithm solver in order to create a realistic solution.

**We have created the following constraints:**
- A class can't be in two lessons at the same time
- A subject can't be in two lessons at the same time;
    - Assuming that there is only a teacher for each subject.
- A room can't be in two lessons at the same, unless it's online;
    - Two classes can have an online lesson at the same time, but two classes can't have two lessons in the same presential room.
- A presential lesson can't be booked in the same day of an online lesson (per class);
    - This reduces trips to IPCA, and because of this constraint, there will be an "online day", plus the free day, so the class will only have to go to IPCA 3 days per week.
- A lesson can't be booked more than 4 hours after or more than 4 hours before another lesson (per class);
    - This ensures that there are no big gaps between lessons.
- There are one or two online lessons per week (per class);
- There is a maximum of tree lessons per day (per class);
- There is a random free day per week (per class);
    - This reduces trips to IPCA.
- There are two lessons of each subject per week (per class);
    - This ensures that each class has the same number of lessons of each subject.
- There are two to four lessons in a specific classroom (per class):
    - This uses the "favourite room".

In [ ]:
# Problem' constraints / restrictions
restrictions = [
    #Constraint(domain.keys(), all_diff_constraint)
]


# Assigning constraints to lessons
# These two fors are created so we ensure that all Lessons are compared to each other
# L1 -> L2, L1 -> L3, L1 -> L4 , etc..
# L2 -> L3, L2 -> L4, etc..
# etc.. 
for x in range (0, (len(classes)*10)):
    for y in range (x+1, (len(classes)*10)):
        # L1.c = 1, L2.c = 1
        # L1.w = 2, L2.w = 2
        # [L1.st, L1.st+L1.d[ != L2.st   ou (L2.st >= L1.st+L1.d        ou L2.st + L2.d <= L1.st)
        # a class can't be in two lessons at the same time
        constraint_class_lesson_at_same_time = Constraint((f'L{x}.c', f'L{y}.c', f'L{x}.w', f'L{y}.w', f'L{x}.st', f'L{y}.st', f'L{x}.d', f'L{y}.d'), lambda lxc, lyc, lxw, lyw, lxst, lyst, lxd, lyd: (lxst >= (lyst + lyd) or lyst >= (lxst + lxd)) if(lxc == lyc and lxw == lyw) else True)


        # L1.su = 1, L2.su = 1
        # L1.w = 2, L2.w = 2
        # [L1.st, L1.st+L1.d[ != L2.st   ou (L2.st >= L1.st+L1.d        ou L2.st + L2.d <= L1.st)
        # a subject (assuming that a teacher is assigned to a subject) can't be in two lessons at the same time
        constraint_subject_lesson_at_same_time = Constraint((f'L{x}.su', f'L{y}.su', f'L{x}.w', f'L{y}.w', f'L{x}.st', f'L{y}.st', f'L{x}.d', f'L{y}.d'), lambda lxsu, lysu, lxw, lyw, lxst, lyst, lxd, lyd: (lxst >= (lyst + lyd) or lyst >= (lxst + lxd)) if(lxsu == lysu and lxw == lyw) else True)
        

        # a room (except online), can't be in two lessons at the same time
        constraint_room_lesson_at_same_time = Constraint((f'L{x}.r', f'L{y}.r', f'L{x}.w', f'L{y}.w', f'L{x}.st', f'L{y}.st', f'L{x}.d', f'L{y}.d'), lambda lxr, lyr, lxw, lyw, lxst, lyst, lxd, lyd: (lxst >= (lyst + lyd) or lyst >= (lxst + lxd)) if(lxr == lyr and lxw == lyw and lxr != 0) else True)


        # in the same day, a class can have either online or presencial lessons
        # by doing this, we reduce the number of trips to IPCA
        constraint_cant_book_presencial_on_same_day_of_online = Constraint((f'L{x}.c', f'L{y}.c', f'L{x}.w', f'L{y}.w', f'L{x}.r', f'L{y}.r'), lambda lxc, lyc, lxw, lyw, lxr, lyr : (lyr == 0 and lxr == 0) or (lyr != 0 and lxr != 0) if(lxc == lyc and lxw == lyw) else True)


        # this constraint assures that there are no big gaps between lessons
        constraint_no_big_gaps_between_classes = Constraint((f'L{x}.c', f'L{y}.c', f'L{x}.w', f'L{y}.w', f'L{x}.st', f'L{y}.st', f'L{x}.d', f'L{y}.d'), lambda lxc, lyc, lxw, lyw, lxst, lyst, lxd, lyd: (lyst >= (lxst + lxd) and lyst <= (lxst + lxd*2)) or (lxst >= (lyst + lyd) and lxst <= (lyst + lyd*2)) if(lxc == lyc and lxw == lyw) else True)        
    
        
        # adding previous created restrictions
        restrictions.append(constraint_class_lesson_at_same_time)
        restrictions.append(constraint_subject_lesson_at_same_time)
        restrictions.append(constraint_room_lesson_at_same_time)
        restrictions.append(constraint_cant_book_presencial_on_same_day_of_online) 
        restrictions.append(constraint_no_big_gaps_between_classes)
        



# in this section, "dynamic" constraints have been created
# the previously created ones can't satisfy some restrictions, so we had to create this "dynamic" function constraints


# each class has one to two online lessons per week
def constraint_one_to_two_online_lessons(*r_list):
    if r_list.count(0) == 1 or r_list.count(0) == 2:
        return True
    else:
        return False 


# each class has a maximum of 3 lessons per day
def constraint_tree_lessons_per_day(*w_list):
    for x in range(2,7):
        if (w_list.count(x) > 3):
            return False
    return True


# each class has a random free day (day without lessons) per week, this reduces trips to IPCA
def constraint_random_free_day_per_week(*w_list):
    random_day = w_list[-1]
    w_tuple_converted_to_list = list(w_list)
    w_tuple_converted_to_list.pop()

    
    if (w_tuple_converted_to_list.count(random_day) > 0):
        return False
    return True


# each class has two lessons of each subject per week (5 subjects * 2 lessons = 10 lessons)
def constraint_two_lessons_of_each_subject_per_week(*su_list):
    for x in subjects:
        if (su_list.count(x) != 2):
            return False
    return True


# each class has two to four lessons in a specific classroom
def constraint_two_to_four_lessons_in_specific_classroom(*r_list):
    favourite_room = r_list[-1]
    r_tuple_converted_to_list = list(r_list)
    r_tuple_converted_to_list.pop()

    if r_list.count(favourite_room) >= 2 and r_list.count(favourite_room) <= 4:
        return True
    return False


# adding previous function constraints to restrictions array
for el in classes:
    one_to_two_online_lessons_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "r")), constraint_one_to_two_online_lessons)
    restrictions.append(one_to_two_online_lessons_constraint)

    tree_lessons_per_day_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "w")), constraint_tree_lessons_per_day)
    restrictions.append(tree_lessons_per_day_constraint)

    random_free_day_per_week_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "w") + get_day_from_class(el, "fd")), constraint_random_free_day_per_week)
    restrictions.append(random_free_day_per_week_constraint)
    
    two_lessons_of_each_subject_per_week_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "su")), constraint_two_lessons_of_each_subject_per_week)
    restrictions.append(two_lessons_of_each_subject_per_week_constraint)

    two_to_four_lessons_in_specific_classroom_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "r") + get_day_from_class(el, "fr")), constraint_two_to_four_lessons_in_specific_classroom)
    restrictions.append(two_to_four_lessons_in_specific_classroom_constraint)


## Algorithm Execution
To execute this algorithm, we use functions and classes from AIMA code.  
In this case, we use NaryCSP, that groups our domain and restrictions as a Constraint Satisfaction Problem; and the ac_solver, that solves this CSP using an arc consistency algorithm with domain splitting.

In [ ]:

class_scheduling = NaryCSP(domain, restrictions)
# print(class_scheduling.variables) # tests
# dict_solver = ac_search_solver(class_scheduling, arc_heuristic=sat_up) # tests
dict_solver = ac_solver(class_scheduling, arc_heuristic=sat_up)
print(dict_solver)
print("--- %s seconds ---" % (time.time() - start_time))

# Full algorithm

In [ ]:
from lessons import Lesson
from csp import *
from my_utils import *
import time

# start_time of execution
start_time = time.time()

# classes used in the algorithm, the name is irrelevant since only the numbers will be used
classes = {
    1: "LESI",
    2: "LESI-PL"
}

# subjects used in the algorithm, the algorithm accepts, at the moment, 5 subjects, the name is irrelevant
subjects =  {
    1: "Mobile",
    2: "Programming",
    3: "Artificial Intelligence",
    4: "Data Structures",
    5: "APIs"
}

# rooms used in the algorithm, the name is irrelevant but ROOM 0 = online!
rooms = {
    0: "Online",
    1: "Sala L",
    2: "Sala T",
    3: "Sala N"
}

# each class has 2 to 4 lessons in a specific classroom
# to distribute "favourite" rooms per classes, we created a room_usages dictionary
# whenever we assign a room as a "favourite" room, the "usage" value of that room is incremented
# this ensures that rooms are assigned as favourite rooms in a balanced way
rooms_usages = {}
for room in rooms.keys():
    if room != 0:
        rooms_usages[room] = 0


# each class has 10 lessons per week
lessons_list = []
for x in range (len(classes)*10):
    # Lesson is a class that was created when we started coding this project
    # It is kinda irrelevant / unused at the moment
    new_l = Lesson(None, None, None, None, None, None)
    lessons_list.append(new_l)


# Domain defines the domain of each variable
domain = {}

# each class has 10 lessons
# Lesson 0-9 corresponds to Class 1
# Lesson 10-19 corresponds to Class 2...
# etc.
aux = 0
aux_final = 10
for x in classes:
    free_day = random.randint(2,6)

    room = least_used_room(rooms_usages)
    rooms_usages[room]+=1

    domain.update({f'L{aux}.fd': {free_day}}) # each class has a random day without lessons
    domain.update({f'L{aux}.fr': {room}}) # each class has a "favourite" room, where it has 2 to 4 lessons per week
    while aux != aux_final: 
        domain.update({f'L{aux}.c': {x}}) # assigning lessons to classes
        aux+=1
    
    aux = aux_final
    aux_final = aux+10


for index, list_el in enumerate(lessons_list):
    domain.update({f'L{index}.su': set(range(1,len(subjects)+1))}) # subjects domain assign
    domain.update({f'L{index}.d': {2}}) # duration domain assign 
    domain.update({f'L{index}.w': set(range(2,7))}) # weekday domain assign
    domain.update({f'L{index}.st': set(range(8,17))}) # start time domain assign
    domain.update({f'L{index}.r': set(range(0,len(rooms)+1))}) # rooms domain assign

# Problem' constraints / restrictions
restrictions = [
    #Constraint(domain.keys(), all_diff_constraint)
]


# Assigning constraints to lessons
# These two fors are created so we ensure that all Lessons are compared to each other
# L1 -> L2, L1 -> L3, L1 -> L4 , etc..
# L2 -> L3, L2 -> L4, etc..
# etc.. 
for x in range (0, (len(classes)*10)):
    for y in range (x+1, (len(classes)*10)):
        # L1.c = 1, L2.c = 1
        # L1.w = 2, L2.w = 2
        # [L1.st, L1.st+L1.d[ != L2.st   ou (L2.st >= L1.st+L1.d        ou L2.st + L2.d <= L1.st)
        # a class can't be in two lessons at the same time
        constraint_class_lesson_at_same_time = Constraint((f'L{x}.c', f'L{y}.c', f'L{x}.w', f'L{y}.w', f'L{x}.st', f'L{y}.st', f'L{x}.d', f'L{y}.d'), lambda lxc, lyc, lxw, lyw, lxst, lyst, lxd, lyd: (lxst >= (lyst + lyd) or lyst >= (lxst + lxd)) if(lxc == lyc and lxw == lyw) else True)


        # L1.su = 1, L2.su = 1
        # L1.w = 2, L2.w = 2
        # [L1.st, L1.st+L1.d[ != L2.st   ou (L2.st >= L1.st+L1.d        ou L2.st + L2.d <= L1.st)
        # a subject (assuming that a teacher is assigned to a subject) can't be in two lessons at the same time
        constraint_subject_lesson_at_same_time = Constraint((f'L{x}.su', f'L{y}.su', f'L{x}.w', f'L{y}.w', f'L{x}.st', f'L{y}.st', f'L{x}.d', f'L{y}.d'), lambda lxsu, lysu, lxw, lyw, lxst, lyst, lxd, lyd: (lxst >= (lyst + lyd) or lyst >= (lxst + lxd)) if(lxsu == lysu and lxw == lyw) else True)
        

        # a room (except online), can't be in two lessons at the same time
        constraint_room_lesson_at_same_time = Constraint((f'L{x}.r', f'L{y}.r', f'L{x}.w', f'L{y}.w', f'L{x}.st', f'L{y}.st', f'L{x}.d', f'L{y}.d'), lambda lxr, lyr, lxw, lyw, lxst, lyst, lxd, lyd: (lxst >= (lyst + lyd) or lyst >= (lxst + lxd)) if(lxr == lyr and lxw == lyw and lxr != 0) else True)


        # in the same day, a class can have either online or presencial lessons
        # by doing this, we reduce the number of trips to IPCA
        constraint_cant_book_presencial_on_same_day_of_online = Constraint((f'L{x}.c', f'L{y}.c', f'L{x}.w', f'L{y}.w', f'L{x}.r', f'L{y}.r'), lambda lxc, lyc, lxw, lyw, lxr, lyr : (lyr == 0 and lxr == 0) or (lyr != 0 and lxr != 0) if(lxc == lyc and lxw == lyw) else True)


        # this constraint assures that there are no big gaps between lessons
        constraint_no_big_gaps_between_classes = Constraint((f'L{x}.c', f'L{y}.c', f'L{x}.w', f'L{y}.w', f'L{x}.st', f'L{y}.st', f'L{x}.d', f'L{y}.d'), lambda lxc, lyc, lxw, lyw, lxst, lyst, lxd, lyd: (lyst >= (lxst + lxd) and lyst <= (lxst + lxd*2)) or (lxst >= (lyst + lyd) and lxst <= (lyst + lyd*2)) if(lxc == lyc and lxw == lyw) else True)        
    
        
        # adding previous created restrictions
        restrictions.append(constraint_class_lesson_at_same_time)
        restrictions.append(constraint_subject_lesson_at_same_time)
        restrictions.append(constraint_room_lesson_at_same_time)
        restrictions.append(constraint_cant_book_presencial_on_same_day_of_online) 
        restrictions.append(constraint_no_big_gaps_between_classes)
        



# in this section, "dynamic" constraints have been created
# the previously created ones can't satisfy some restrictions, so we had to create this "dynamic" function constraints


# each class has one to two online lessons per week
def constraint_one_to_two_online_lessons(*r_list):
    if r_list.count(0) == 1 or r_list.count(0) == 2:
        return True
    else:
        return False 


# each class has a maximum of 3 lessons per day
def constraint_tree_lessons_per_day(*w_list):
    for x in range(2,7):
        if (w_list.count(x) > 3):
            return False
    return True


# each class has a random free day (day without lessons) per week, this reduces trips to IPCA
def constraint_random_free_day_per_week(*w_list):
    random_day = w_list[-1]
    w_tuple_converted_to_list = list(w_list)
    w_tuple_converted_to_list.pop()

    
    if (w_tuple_converted_to_list.count(random_day) > 0):
        return False
    return True


# each class has two lessons of each subject per week (5 subjects * 2 lessons = 10 lessons)
def constraint_two_lessons_of_each_subject_per_week(*su_list):
    for x in subjects:
        if (su_list.count(x) != 2):
            return False
    return True


# each class has two to four lessons in a specific classroom
def constraint_two_to_four_lessons_in_specific_classroom(*r_list):
    favourite_room = r_list[-1]
    r_tuple_converted_to_list = list(r_list)
    r_tuple_converted_to_list.pop()

    if r_list.count(favourite_room) >= 2 and r_list.count(favourite_room) <= 4:
        return True
    return False


# adding previous function constraints to restrictions array
for el in classes:
    one_to_two_online_lessons_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "r")), constraint_one_to_two_online_lessons)
    restrictions.append(one_to_two_online_lessons_constraint)

    tree_lessons_per_day_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "w")), constraint_tree_lessons_per_day)
    restrictions.append(tree_lessons_per_day_constraint)

    random_free_day_per_week_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "w") + get_day_from_class(el, "fd")), constraint_random_free_day_per_week)
    restrictions.append(random_free_day_per_week_constraint)
    
    two_lessons_of_each_subject_per_week_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "su")), constraint_two_lessons_of_each_subject_per_week)
    restrictions.append(two_lessons_of_each_subject_per_week_constraint)

    two_to_four_lessons_in_specific_classroom_constraint = Constraint(tuple(get_only_list_of_attribute_from_class(el, "r") + get_day_from_class(el, "fr")), constraint_two_to_four_lessons_in_specific_classroom)
    restrictions.append(two_to_four_lessons_in_specific_classroom_constraint)




class_scheduling = NaryCSP(domain, restrictions)
# print(class_scheduling.variables) # tests
# dict_solver = ac_search_solver(class_scheduling, arc_heuristic=sat_up) # tests
dict_solver = ac_solver(class_scheduling, arc_heuristic=sat_up)
print(dict_solver)
print("--- %s seconds ---" % (time.time() - start_time))

# GitHub repository
https://github.com/duartemelo/IA_Class_Scheduling